In [1]:
import re
from collections import defaultdict
from math import inf as INFINITY
from itertools import product, combinations

In [2]:
filename = 'input.txt'
with open(filename, 'r+') as f:
    lines = f.readlines()
lines = [re.split('[\\s=;,]+', x.strip()) for x in lines]
G = {line[1]:line[10:] for line in lines} # graph
F = {line[1]:int(line[5]) for line in lines if int(line[5]) != 0} # flows exluding valves with flow 0
valves = set(F.keys()) # only visit valves that have flow > 0

def score(flow_rates, chosen_valves):
    result = 0
    for valve, time_left in chosen_valves.items():
        result += flow_rates[valve] * time_left
    return result

def floyd_warshall(graph):
    # Floyd-Warshall algorithm
    # returns distance[Dict[Dict]]
    # distance['AA']['BB'] -> minimum distance from valve AA to BB
    distance = defaultdict(lambda:defaultdict(lambda: INFINITY))
    for cur_node, nodes in graph.items():
        distance[cur_node][cur_node] = 0

        for node in nodes:
            distance[cur_node][node] = 1
            distance[node][node] = 0

    for a,b,c in product(graph, graph, graph):
        bc, ba, ac = distance[b][c], distance[b][a], distance[a][c]
        if ba + ac < bc:
            distance[b][c] = ba + ac

    return distance

def floyd2(G):
    T = {x: {y: 1 if y in G[x] else float('+inf') for y in G} for x in G}
    for k in T:
        for i in T:
            for j in T:
                T[i][j] = min(T[i][j], T[i][k]+T[k][j])
    return T

In [7]:
def solutions(distance, valves, time = 30, cur = 'AA', chosen = {}):
    # For all the valves we can currenntly choose:
    for nxt in valves:
        # 1 min plus to open the valve
        new_time = time - (distance[cur][nxt] + 1)
        # Stop condition for recursion - out of budget
        if new_time <= 0:
            continue
        # dict union python 3.9
        # new_chosen = chosen | {nxt:new_time}
        new_chosen = {**chosen, **{nxt:new_time}}
        # remove nxt from set of valves
        new_valves = valves - {nxt}
        yield from solutions(distance, new_valves, new_time, nxt, new_chosen)
    yield chosen

In [8]:
distance = floyd_warshall(G)
best = 0
for choice in solutions(distance, valves):
    cur = score(F, choice)
    if cur > best:
        best = cur
print(best)

1595


In [13]:
# part II

maxscore = defaultdict(int)
distance = floyd2(G)
choices = list(solutions(distance, valves, time = 26))

for choice in choices:
    k = frozenset(choice)
    s = score(F, choice)

    if s > maxscore[k]:
        maxscore[k] = s

In [17]:
choices[1]

{'ZB': 20, 'CE': 14, 'CD': 6, 'TM': 2}

In [10]:
total2 = max(v1+v2 for k1, v1 in maxscore.items() for k2, v2 in maxscore.items() if not k1 & k2)

In [201]:
best = 0 # correct is 2189 (why?!)
for (s1, score1), (s2, score2) in combinations(maxscore.items(), 2):
    #print(s1, s2)
    if len(s1.intersection(s2)) != 0 or (len(s1) == 0) or (len(s2) == 0):
        continue

    cur = score1 + score2
    if cur > best:
        best = cur

In [11]:
best = max(sa + sb for (a, sa), (b, sb) in combinations(maxscore.items(), 2) if not a & b)

In [12]:
best

2189